In [1]:
import os
os.getcwd()

'/Users/macbookpro/Documents/171001-181001-no-qt/clean analysis'

In [2]:
import sys
import json
import re
import numpy as np
import pandas as pd

tweetfile = 'cleaned-dataset.csv'

all_tweets = pd.read_csv(tweetfile)

In [3]:
all_tweets.count()

created_at                 94266
user_screen_name           94266
text                       94263
favorite_count             94266
retweet_count              94266
in_reply_to_screen_name    34799
user_followers_count       94266
user_friends_count         94266
place                      91717
dtype: int64

In [4]:
all_tweets.head(5)

,created_at,user_screen_name,text,favorite_count,retweet_count,in_reply_to_screen_name,user_followers_count,user_friends_count,place
0,Tue Dec 12 00:09:18 +0000 2017,CodyMcDowall,im dispute climate change weve gotta stop clai...,3,0,NaN,176,113,"Orange Park, FL"
1,Tue Sep 11 22:41:30 +0000 2018,mebeandreaaolco,many american kill climatechangeisreal remembe...,1,0,realDonaldTrump,2882,4959,"Twentynine Palms, CA"
2,Tue Nov 21 03:04:45 +0000 2017,hanjangho76,there nothing need fear stand together right s...,0,0,NaN,3864,5001,"San Jose, CA"
3,Sat Jan 06 21:17:16 +0000 2018,AlixStroiazzo,look new crypto currency going revolutionize p...,0,0,NaN,44,26,"Manhattan, NY"
4,Tue Nov 21 02:51:58 +0000 2017,hanjangho76,beyond blessed find right side history rest hi...,0,0,NaN,3864,5001,"San Jose, CA"


In [5]:
all_tweets.shape

(94266, 9)

In [6]:
# Export edges from Replies
replies = all_tweets[all_tweets['in_reply_to_screen_name'].notnull()]

edges = replies[['user_screen_name', 'in_reply_to_screen_name','created_at']]
edges.columns = ['Source', 'Target', 'Weight']

In [7]:
strengthLevel = 3  # Network connection strength level: the number of times in total each of the tweeters responded to or mentioned the other.
                   # If you have 1 as the level, then all tweeters who mentioned or replied to another at least once will be displayed. But if you have 5, only those who have mentioned or responded to a particular tweeter at least 5 times will be displayed, which means that only the strongest bonds are shown.

edges2 = edges.groupby(['Source','Target'])['Weight'].count()
edges2 = edges2.reset_index()
edges2 = edges2[edges2['Weight'] >= strengthLevel]

In [8]:
# Export nodes from the edges and add node attributes for both Sources and Targets.

users = all_tweets[['user_screen_name','user_followers_count', 'user_friends_count']]
users = users.sort_values(['user_screen_name','user_followers_count'], ascending=[True, False])
users = users.drop_duplicates(['user_screen_name'], keep='first') 

ids = edges2['Source'].append(edges2['Target']).to_frame()
ids['Label'] = ids
ids.columns = ['user_screen_name', 'Label']
ids = ids.drop_duplicates(['user_screen_name'], keep='first') 
nodes = pd.merge(ids, users, on='user_screen_name', how='left')

In [10]:
print(nodes.shape)
print(edges.shape)

(36466, 4)
(34799, 2)


In [11]:
# change column names for Kumu import (Run this when using Kumu)
edges2.columns = ['Source','Target','Weight']
nodes.columns = ['Id', 'Label', 'user_followers_count', 'user_friends_count']

In [12]:
nodes

,Id,Label,user_followers_count,user_friends_count
0,mebeandreaaolco,mebeandreaaolco,2882.0,4959.0
1,RicVaDude,RicVaDude,44231.0,44798.0
2,hockendougal,hockendougal,1622.0,1531.0
3,HeestermanNeil,HeestermanNeil,144.0,168.0
4,JocelynRenee81,JocelynRenee81,35.0,312.0
...,...,...,...,...
36461,infpwriter,infpwriter,NaN,NaN
36462,WiFiJanitor,WiFiJanitor,NaN,NaN
36463,Ward_L_Shearer,Ward_L_Shearer,NaN,NaN
36464,melulater,melulater,NaN,NaN


In [13]:
edges2

,Source,Target
1,mebeandreaaolco,realDonaldTrump
8,RicVaDude,MrsMalindo
10,hockendougal,aepasek
11,HeestermanNeil,Pembina
17,JocelynRenee81,realDonaldTrump
...,...,...
94253,ShowUpAmerica,SenSanders
94257,RJDownard,RealSaavedra
94262,StuartinTrosley,jdcnorden
94263,EvansNjomo,EvansNjomo


In [15]:
# Export nodes and edges to csv files
nodes.to_csv('nodes_replies_tweet6.csv', encoding='utf-8', index=False)
edges2.to_csv('edges_replies_tweet6.csv', encoding='utf-8', index=False)